In [1]:
import wave
import threading
import tkinter
import tkinter.filedialog
import tkinter.messagebox
import pyaudio
from timeit import Timer
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pygame
import os
import os.path
import shutil
from PIL import Image,ImageTk

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
CHUNK_SIZE = 1024
CHANNELS = 2
FORMAT = pyaudio.paInt16
RATE = 44100
RECORD_SECONDS = 5
fileName = None
allowRecording = False
uid_to_human = {}
#imgdir = None

def timecontrol():
    global allowRecording
    allowRecording = False
    lbStatus['text'] = '准备就绪'

def id_to_string(node_id):
    if node_id not in uid_to_human:
        return ''
    return uid_to_human[node_id]

def record():
    #global allowRecording
    global fileName, timer
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,channels=CHANNELS,rate=RATE,input=True,frames_per_buffer=CHUNK_SIZE)
    wf = wave.open(fileName, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    
    #timer = threading.Timer(20,timecontrol)
    #timer.start()
    #Timer(20,timecontrol).start()
    while allowRecording:
        timer = threading.Timer(21,timecontrol)
        timer.start()
        #从录音识别读取数据，直接写入文件
        data = stream.read(CHUNK_SIZE)
        wf.writeframes(data)
    wf.close()
    stream.stop_stream();
    stream.close()
    p.terminate()
    fileName = None
#创建tkinter应用程序
root = tkinter.Tk()
root.title("录音")
root.geometry('435x500+200+300')
root.resizable(False,False)

#识别按钮
def predict():
    lines = tf.gfile.GFile('retrain/output_labels.txt').readlines()
    #一行一行读数据
    for uid,line in enumerate(lines):
        line = line.strip('\n')
        uid_to_human[uid] = line
    
    #创建一个图来存放google训练好的模型
    with tf.gfile.FastGFile('retrain/output_graph.pb','rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')
    with tf.Session() as sess:
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
        #选择图片
        image_path = tkinter.filedialog.askopenfilename()
        image_data = tf.gfile.FastGFile(image_path, 'rb').read()
        
        predictions = sess.run(softmax_tensor,{'DecodeJpeg/contents:0':image_data})
        predictions = np.squeeze(predictions)
        #打印图片路径及名称
        #print(image_path)
        lbStatus['text'] = '准备就绪'
        #显示图片
        #img = Image.open(image_path)
        # plt.imshow(img)
        # plt.axis('off')
        # plt.show
        
        #排序
        top_k = predictions.argsort()[::-1]
        print(top_k)
        i = 0
        for node_id in top_k:
            #获取分类名称
            human_string = id_to_string(node_id)
            #获取该分类的置信度
            score = predictions[node_id]
            #print('%s (score = %.5f)' % (human_string, score))
            round(score,5)
            score = str(score)
            result = human_string + ' (score = ' + score + ')'  
            if i == 0:
                lbStatus2['text'] = result
            if i == 1:
                lbStatus3['text'] = result
            if i == 2:
                lbStatus4['text'] = result
            
            i += 1
                
        print()
    
    lbStatus['text'] = '准备就绪'
    root.update_idletasks()
btnStop = tkinter.Button(root,text='识别情绪',command=predict)
btnStop.place(x=315,y=460,width=100,height=20)

lbStatus = tkinter.Label(root,text='准备就绪',anchor='w',fg='green')
lbStatus.place(x=190,y=420,width=200,height=20)

lbStatus1 = tkinter.Label(root,text='识别结果',anchor='w',fg='green')
lbStatus1.place(x=190,y=320,width=200,height=20)

lbStatus2 = tkinter.Label(root,text='',anchor='w',fg='red')
lbStatus2.place(x=140,y=340,width=250,height=20)

lbStatus3 = tkinter.Label(root,text='',anchor='w',fg='red')
lbStatus3.place(x=140,y=360,width=250,height=20)

lbStatus4 = tkinter.Label(root,text='',anchor='w',fg='red')
lbStatus4.place(x=140,y=380,width=250,height=20)



#开始按钮
def start():
    global allowRecording,fileName
    fileName = tkinter.filedialog.asksaveasfilename(filetypes=[('未压缩波形文件','*.wav')])
    
    if not fileName:
        return
    if not fileName.endswith('.wav'):
        fileName = fileName+'.wav'
    allowRecording = True
    lbStatus['text'] = '正在录音...'
    threading.Thread(target=record).start()
btnStart = tkinter.Button(root,text='开始录音',command=start)
btnStart.place(x=20,y=460,width=100,height=20)

#生成时频图按钮
def creategraph():
    
    global allowRecording,fileName
    fileName = tkinter.filedialog.askopenfilename()
    fileNamefl = str(fileName)
    fileNamefl.replace("/","\\")
    
    (filepath,tempfilename) = os.path.split(fileName)
    (filename,extension) = os.path.splitext(tempfilename)
    root.update_idletasks()
    lbStatus['text'] = '正在生成频谱图'
    #rootdir = fileName
    #list = os.listdir(rootdir)
    i = int(0)
    #打开wav文件，文件路径根据需要做修改
    f = wave.open(fileNamefl)
    params = f.getparams()
    nchannels,sampwidth,framerate,nframes = params[:4]
    #得到的数据是字符串，需要将其转换成int型
    strData = f.readframes(nframes)
    waveData = np.fromstring(strData,dtype=np.int16)
    waveData = np.reshape(waveData,[nframes,nchannels]).T
    f.close()
    #绘制频谱
    plt.specgram(waveData[0],Fs = framerate,scale_by_freq = True,sides = 'default')
    plt.ylabel('Frequency(kHz)')
    plt.xlabel('Time(s)')
    plt.savefig("E:/TensorflowProject/temp/"+filename+".jpg")
    lbStatus['text'] = '准备就绪'
    #tkinter.updata()
    #imgdir = "E:/TensorflowProject/temp/"+filename+".jpg"
    im = Image.open("E:/TensorflowProject/temp/"+filename+".jpg")
    img = ImageTk.PhotoImage(im)
    imLabel = tkinter.Label(root,image=img,compound='center')
    imLabel.configure(image = img)
    imLabel.grid(column=0,row=0,sticky='W')
    #label_img.configure(image = img)
    #root.update_idletasks()
    root.mainloop()
    #plt.show()
    print(fileName)
btnCreate = tkinter.Button(root,text='生成频谱图',command=creategraph)
btnCreate.place(x=167,y=460,width=100,height=20)


#关闭程序时检查是否正在录制
def closeWindow():
    if allowRecording:
        tkinter.messagebox.showerror('正在录制','请先停止录制')
        return
    root.destroy()
root.protocol('WM_DELETE_WINDOW',closeWindow)
root.mainloop()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:169: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


Instructions for updating:
Use tf.gfile.GFile.
[2 0 1]

